# TorchAudio 维护期速览与实践指南

> 汇总 TorchAudio 进入维护阶段后的要点，梳理主要模块（I/O、functional、transforms、datasets、models、pipelines 等）以及 TorchCodec 迁移提示，并给出可本地运行的最小示例。若当前环境未安装 `torchaudio`，可先阅读概念部分再行安装。

## 保持更新：维护阶段与 TorchCodec 合并
- **维护公告**：自 2.8 起进入维护期，2.9 移除在 2.8 已弃用的 API。
- **编解码迁移**：音视频解码/编码能力已并入 TorchCodec；`torchaudio.load/save` 现为 `load_with_torchcodec/save_with_torchcodec` 的别名。
- **迁移建议**：新项目优先直接使用 TorchCodec API，对旧有代码可先锁定 `torchaudio<2.9` 或 `torchdata` 仓库中的公告细则（<https://github.com/pytorch/audio/issues/3902>）。

In [ ]:
# 检查 torchaudio / torchcodec 可用性
try:
    import torchaudio
    print('torchaudio 版本:', torchaudio.__version__)
    print('torchaudio 路径:', torchaudio.__file__)
    has_torchcodec = False
    try:
        import torchcodec
        has_torchcodec = True
    except ImportError:
        pass
    print('TorchCodec 可用:', has_torchcodec)
except ImportError as err:
    print('未安装 torchaudio ->', err)
    print('安装示例: pip install torchaudio --extra-index-url https://download.pytorch.org/whl/cu118')

## TorchAudio 教程索引回顾
- **ASR / CTC / Forced Alignment**：涵盖 GPU Beam Search、CTC Alignment、多语言对齐。
- **Pipelines / Preprocessing**：音频重采样、数据增广、特征提取。
- **Source Separation / Enhancement / TTS**：Demucs、SQUIM、Tacotron2 等完整流水线。
- **wav2vec2 系列**：自监督特征、ASR、Forced Alignment 教程。
> 官方教程按类别划分，可在文档站点快速定位（Speech、TTS、ASR 等）。

## 引用 TorchAudio
- 2023 版：`TorchAudio 2.1: Advancing speech recognition ...` (arXiv:2310.17864)
- 2021 版：`TorchAudio: Building Blocks for Audio and Speech Processing` (arXiv:2110.15018)
- **BibTeX**：
  ```bibtex
  @misc{hwang2023torchaudio, ...}
  @article{yang2021torchaudio, ...}
  ```
在论文或产品中使用 TorchAudio 时，请根据实际版本引用相应论文。

## 特性徽章与支持矩阵
- **设备**：`CPU`、`CUDA` 徽章表示已通过自动化测试验证，可直接在对应设备上运行。
- **属性**：`Autograd`、`TorchScript` 徽章分别指示梯度传播与脚本化支持状态。
- **缺失徽章的含义**：可能未测试或不兼容；使用前需自行验证，尤其在梯度或脚本化场景。
- **发布状态**：`Stable`（长期维护）、`Beta`（功能待完善）、`Prototype`（通常需源码安装或运行时开关）。

## 设备 / Autograd / TorchScript 快速示例

In [ ]:
import torch
try:
    import torchaudio
    waveform = torch.randn(1, 16000)
    resampler = torchaudio.transforms.Resample(orig_freq=16000, new_freq=8000)
    resampled = resampler(waveform)
    print('Resample 成功，长度:', resampled.shape[-1])
    if torch.cuda.is_available():
        device = torch.device('cuda')
        resampler = resampler.to(device)
        res_gpu = resampler(waveform.to(device))
        print('CUDA 输出长度:', res_gpu.shape[-1])
except Exception as err:
    print('示例执行失败:', err)

## 顶层模块：`torchaudio`
- **I/O**：`load` / `save` 现为 TorchCodec 别名，推荐逐步迁移到 `torchaudio.load_with_torchcodec` / `torchaudio.save_with_torchcodec`。
- **其他子模块**：`functional`、`transforms`、`datasets`、`models`、`pipelines`、`compliance.kaldi`、`models.decoder` 等提供音频信号处理、模型构建与任务级打包。

In [ ]:
import io
try:
    import torchaudio
    sample_rate = 16000
    duration = 0.5
    t = torch.linspace(0, duration, int(sample_rate * duration))
    sine = torch.sin(2 * torch.pi * 440 * t).unsqueeze(0)

    buffer = io.BytesIO()
    torchaudio.save(buffer, sine, sample_rate)
    buffer.seek(0)
    waveform, sr = torchaudio.load(buffer)
    print('load/save roundtrip -> waveform:', waveform.shape, 'sample_rate:', sr)
except Exception as err:
    print('load/save 示例失败:', err)

## `torchaudio.functional` 常用类别
- **Utility**：`amplitude_to_DB`、`resample`、`mask_along_axis`、`preemphasis` 等。
- **Forced Alignment**：`forced_align`（已弃用）、`merge_tokens`、`TokenSpan`。
- **Filtering**：`biquad`、`bandpass_biquad`、`lfilter`、`overdrive` 等效果器。
- **Feature Extraction**：`spectrogram`、`griffinlim`、`pitch_shift`、`detect_pitch_frequency` 等。
- **多通道**：`psd`、`mvdr_weights_*`、`rtf_*`、`apply_beamforming`。
- **Loss / Metric**：`rnnt_loss`（弃用）、`edit_distance`。
> functional 系列偏向函数式 API，可配合 `torch.autograd` 使用。

In [ ]:
try:
    import torchaudio.functional as AF
    waveform = torch.randn(1, 32000)
    spec = AF.spectrogram(waveform, pad=0, window=torch.hann_window(400), n_fft=400, hop_length=160, win_length=400)
    mel_fb = AF.melscale_fbanks(201, 40, sample_rate=16000)
    mel_spec = torch.matmul(spec.squeeze(0).transpose(0, 1), mel_fb.T)
    print('Spectrogram 形状:', spec.shape, 'Mel 形状:', mel_spec.shape)
except Exception as err:
    print('functional 示例失败:', err)

## `torchaudio.transforms`：模块化信号处理
- **核心思想**：全部基于 `torch.nn.Module`，可组合或通过 `nn.Sequential` 构造流水线。
- **Utility**：`Resample`、`Fade`、`Vol`、`Speed`、`Loudness`、`AddNoise` 等。
- **Feature Extraction**：`Spectrogram`、`MelSpectrogram`、`MFCC`、`LFCC`、`PitchShift`、`SpectralCentroid`、`Vad`。
- **增广（SpecAugment）**：`FrequencyMasking`、`TimeMasking`、`TimeStretch`。
- **多通道**：`PSD`、`MVDR`、`SoudenMVDR` 等。

In [ ]:
try:
    import torchaudio.transforms as AT
    class FeaturePipeline(torch.nn.Module):
        def __init__(self, sample_rate=16000):
            super().__init__()
            self.resample = AT.Resample(orig_freq=sample_rate, new_freq=8000)
            self.spec = AT.Spectrogram(n_fft=256, power=2)
            self.augment = torch.nn.Sequential(
                AT.TimeStretch(0.9, fixed_rate=True),
                AT.FrequencyMasking(freq_mask_param=10),
                AT.TimeMasking(time_mask_param=15),
            )
            self.mel = AT.MelScale(n_mels=40, sample_rate=8000, n_stft=129)
        def forward(self, waveform):
            x = self.resample(waveform)
            x = self.spec(x)
            x = self.augment(x)
            x = self.mel(x)
            return x
    pipeline = FeaturePipeline()
    feat = pipeline(torch.randn(1, 16000))
    print('Mel 特征形状:', feat.shape)
except Exception as err:
    print('transforms 示例失败:', err)

## `torchaudio.datasets`：快速加载常用语音数据
- 覆盖 `LIBRISPEECH`、`COMMONVOICE`、`GTZAN`、`SPEECHCOMMANDS`、`YESNO` 等经典数据集。
- 均继承自 `torch.utils.data.Dataset`，可直接配合 `DataLoader` 与 `torchdata` 新 API。
- 下载依赖网络；无网络时可通过 `download=False` 仅使用本地缓存。

In [ ]:
try:
    from torchaudio.datasets import YESNO
    dataset = YESNO(root='dataset', download=False)
    if len(dataset) > 0:
        waveform, sample_rate, labels = dataset[0]
        print('YESNO 条目:', waveform.shape, sample_rate, labels)
    else:
        print('YESNO 数据集为空（可能未下载）。')
except Exception as err:
    print('datasets 示例受限:', err)

## `torchaudio.models` 与 `models.decoder`
- **主要架构**：`Conformer`、`ConvTasNet`、`DeepSpeech`、`Emformer`、`HDemucs`、`HuBERT`、`RNNT`、`Tacotron2`、`Wav2Vec2`、`WaveRNN` 等。
- **CTC 解码**：`torchaudio.models.decoder.CTCDecoder` 提供 Beam Search；CUDA 版已弃用。
- **注意**：预训练权重多在 `torchaudio.pipelines` 或 `pipelines` Bundle 中提供。

In [ ]:
try:
    from torchaudio.models import Wav2Vec2Model
    model = Wav2Vec2Model.extractor( # 使用工厂方法构造特征提取器
        extractor_mode='group_norm',
        extractor_conv_layer_config=[(512, 10, 5)]
    )
    dummy = torch.randn(1, 16000)
    out = model(dummy)
    print('Wav2Vec2 特征输出:', out.shape)
except Exception as err:
    print('models 示例失败:', err)

## `torchaudio.compliance.kaldi`
- 提供 Kaldi 对齐的经典特征：`spectrogram`、`fbank`、`mfcc` 等。
- 参数命名与 Kaldi 一致，便于迁移传统工作流。

In [ ]:
try:
    from torchaudio.compliance import kaldi
    waveform = torch.randn(1, 16000)
    mfcc = kaldi.mfcc(waveform, sample_frequency=16000)
    print('Kaldi MFCC 形状:', mfcc.shape)
except Exception as err:
    print('kaldi 示例失败:', err)

## `torchaudio.pipelines`：任务级 Bundle
- **RNNTBundle**：流式 / 非流式 ASR（Emformer-RNNT）。
- **Wav2Vec2Bundle / Wav2Vec2ASRBundle / Wav2Vec2FABundle**：自监督特征、ASR、Forced Alignment。
- **Tacotron2TTSBundle**：文本到语音（支持 WaveRNN / GriffinLim Vocoder）。
- **SourceSeparationBundle**：音源分离（ConvTasNet、HDemucs）。
- **SQUIM Bundles**：客观 / 主观语音质量评估。

In [ ]:
try:
    from torchaudio.pipelines import WAV2VEC2_BASE
    bundle = WAV2VEC2_BASE
    model = bundle.get_model()
    model.eval()
    feature_dim = model.extract_features(torch.randn(1, 16000))[0].shape
    print('WAV2VEC2_BASE 特征维度:', feature_dim)
except Exception as err:
    print('pipelines 示例失败（可能未下载权重或无网络）:', err)

## 维护期总结与实践建议
1. **API 审查**：确认项目依赖的功能是否在 2.8 被弃用；必要时锁定旧版本或切换替代实现。
2. **TorchCodec 迁移**：针对音频 I/O 尽早过渡至 TorchCodec 直接接口，避免未来别名移除导致破坏。
3. **特性验证**：在 Autograd / TorchScript 场景下务必结合徽章信息进行单元测试。
4. **教程复习**：按照官方分类（ASR、TTS、增广等）逐一试跑，确保对维护期保留能力心中有数。

## 参考链接
- TorchAudio 官方文档：<https://pytorch.org/audio/stable/>
- TorchCodec 项目：<https://github.com/pytorch/codec>
- 维护公告 Issue：<https://github.com/pytorch/audio/issues/3902>
- TorchAudio 论文引用：arXiv:2310.17864 / 2110.15018